<a href="https://colab.research.google.com/github/tiaaburton/Milestone-II-Machine-Learning-Project/blob/main/Playground_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [205]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [206]:
! pip install pyspark

In [207]:
! pip install pyspark[pandas-on-spark] plotly

In [208]:
! pip install chart_studio

In [242]:
import pandas as pd
import numpy as np
import re
import os
import sys

from google.cloud import bigquery
import pyarrow
from datetime import datetime
from tqdm import tqdm

from oauth2client.client import GoogleCredentials
from google.colab import auth, files

import ast
import json
import re
import random

import pyspark
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
import pyspark.ml
from pyspark.ml import *



In [210]:
RANDOM_SEED = 655

In [211]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Binary Buyer Prediction") \
    .getOrCreate()

In [212]:
sample = spark.read.csv('/content/drive/Shareddrives/SIADS - 694-695 Team Drive/processed_file.csv', header=True).withColumnRenamed('_c0','index')

In [213]:
from collections import Counter
core = [col for col in sample.columns if '.' not in col]
additional = [col for col in sample.columns if '.' in col]
second = []
third = []

for col in additional:
  count = Counter(col)
  if count['.'] == 1:
    second.append(col)
  elif count['.'] == 2:
    third.append(col)

In [ ]:
# columns = []
# with open('/content/drive/Shareddrives/SIADS - 694-695 Team Drive/Notebooks/final_columns.txt') as f:
#   for ln in f.readlines():
#     saved = ln.strip().split('\'')
#     columns.append(saved[1])
# print(columns)

In [227]:
processed = pd.read_csv('/content/drive/Shareddrives/SIADS - 694-695 Team Drive/processed_file.csv',index_col=0)

In [228]:
processed.columns

Index(['socialEngagementType', 'hits', 'channelGrouping', 'trafficSource',
       'totals', 'date', 'device', 'fullVisitorId', 'geoNetwork',
       'totals.visits', 'totals.hits', 'totals.pageviews', 'totals.timeOnSite',
       'totals.transactions', 'totals.newVisits', 'totals.screenviews',
       'totals.uniqueScreenviews', 'totals.timeOnScreen',
       'totals.totalTransactionRevenue', 'hits.type', 'geoNetwork.country',
       'trafficSource.source', 'trafficSource.medium',
       'trafficSource.isTrueDirect', 'device.browser',
       'device.operatingSystem', 'device.deviceCategory', 'hits.hour',
       'hits.minute', 'hits.transaction', 'hits.item', 'hits.dataSource',
       'hits.appInfo', 'hits.eCommerceAction', 'totals.bounces',
       'hits.appInfo.screenDepth', 'hits.eCommerceAction.action_type',
       'hits.transaction.transactionRevenue', 'hits.item.productName',
       'hits.item.productCategory', 'hits.item.itemRevenue'],
      dtype='object')

In [221]:
processed.describe()

,date,fullVisitorId,totals.visits,totals.hits,totals.pageviews,totals.timeOnSite,totals.transactions,totals.newVisits,totals.screenviews,totals.uniqueScreenviews,totals.timeOnScreen,totals.totalTransactionRevenue,totals.bounces,hits.appInfo.screenDepth,hits.eCommerceAction.action_type,hits.transaction.transactionRevenue,hits.item.productName,hits.item.productCategory,hits.item.itemRevenue
count,5.010600e+04,5.010600e+04,50106.0,50106.000000,50101.000000,24961.000000,587.000000,39755.0,0.0,0.0,0.0,5.850000e+02,25089.0,50106.0,50106.000000,0.0,0.0,0.0,0.0
mean,2.016220e+07,4.531675e+18,1.0,4.688341,3.881879,245.340251,1.045997,1.0,NaN,NaN,NaN,1.658199e+08,1.0,0.0,0.006067,NaN,NaN,NaN,NaN
std,3.050265e+03,3.065652e+18,0.0,10.101399,7.272584,463.423206,0.302882,0.0,NaN,NaN,NaN,4.323891e+08,0.0,0.0,0.115478,NaN,NaN,NaN,NaN
min,2.016080e+07,4.541792e+13,1.0,1.000000,1.000000,1.000000,1.000000,1.0,NaN,NaN,NaN,2.040000e+06,1.0,0.0,0.000000,NaN,NaN,NaN,NaN
25%,2.016092e+07,1.648222e+18,1.0,1.000000,1.000000,32.000000,1.000000,1.0,NaN,NaN,NaN,3.299000e+07,1.0,0.0,0.000000,NaN,NaN,NaN,NaN
50%,2.016110e+07,4.424537e+18,1.0,2.000000,1.000000,75.000000,1.000000,1.0,NaN,NaN,NaN,6.094000e+07,1.0,0.0,0.000000,NaN,NaN,NaN,NaN
75%,2.016121e+07,7.216722e+18,1.0,4.000000,4.000000,232.000000,1.000000,1.0,NaN,NaN,NaN,1.327000e+08,1.0,0.0,0.000000,NaN,NaN,NaN,NaN
max,2.017013e+07,9.999978e+18,1.0,483.000000,340.000000,9963.000000,5.000000,1.0,NaN,NaN,NaN,7.003500e+09,1.0,0.0,5.000000,NaN,NaN,NaN,NaN


In [229]:
processed = processed.drop(['totals.screenviews','totals.uniqueScreenviews','totals.timeOnScreen',
                            'device','totals','hits','hits.type','trafficSource','geoNetwork'],axis=1)

In [235]:
for c in processed.columns:
  if '.' in c:
    repl = c.replace('.','_')
    processed.rename(columns={c: repl},inplace=True)

In [236]:
processed.head(3)

,socialEngagementType,channelGrouping,date,fullVisitorId,totals_visits,totals_hits,totals_pageviews,totals_timeOnSite,totals_transactions,totals_newVisits,...,hits_dataSource,hits_appInfo,hits_eCommerceAction,totals_bounces,hits_appInfo_screenDepth,hits_eCommerceAction_action_type,hits_transaction_transactionRevenue,hits_item_productName,hits_item_productCategory,hits_item_itemRevenue
0,Not Socially Engaged,Social,20160801,5160807529212499245,1,10,9.0,148.0,NaN,1.0,...,"[None, None, None, None, None, None, None, Non...","[{'name': None, 'version': None, 'id': None, '...","[{'action_type': '0', 'step': 1, 'option': Non...",NaN,0,0,NaN,NaN,NaN,NaN
1,Not Socially Engaged,Organic Search,20160801,558025075151655343,1,2,2.0,203.0,NaN,1.0,...,"[None, None]","[{'name': None, 'version': None, 'id': None, '...","[{'action_type': '0', 'step': 1, 'option': Non...",NaN,0,0,NaN,NaN,NaN,NaN
2,Not Socially Engaged,Direct,20160801,3751823672052015547,1,2,2.0,11.0,NaN,NaN,...,"[None, None]","[{'name': None, 'version': None, 'id': None, '...","[{'action_type': '0', 'step': 1, 'option': Non...",NaN,0,0,NaN,NaN,NaN,NaN


In [237]:
train = processed[processed.columns[~processed.columns.isin(['totals_transactions'])]]
train = train[['date','fullVisitorId','totals_visits','totals_totalTransactionRevenue',
               'totals_bounces','totals_newVisits','totals_hits','totals_timeOnSite',
               'totals_pageviews']]
               
target = processed[processed.columns[processed.columns.isin(['totals_transactions'])]]
target = pd.Categorical(target.iloc[:,0])
target = pd.Series(target.add_categories(0.0).fillna(0.0))

In [ ]:
train.corr()

,date,fullVisitorId,totals_visits,totals_totalTransactionRevenue,totals_bounces,totals_newVisits,totals_hits,totals_timeOnSite,totals_pageviews
date,1.000000,-0.006308,NaN,0.047847,NaN,NaN,-0.001473,0.010124,0.006483
fullVisitorId,-0.006308,1.000000,NaN,0.060864,NaN,NaN,0.003682,-0.006080,0.003720
totals_visits,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
totals_totalTransactionRevenue,0.047847,0.060864,NaN,1.000000,NaN,NaN,0.185626,0.135123,0.184333
totals_bounces,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
totals_newVisits,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
totals_hits,-0.001473,0.003682,NaN,0.185626,NaN,NaN,1.000000,0.623907,0.987265
totals_timeOnSite,0.010124,-0.006080,NaN,0.135123,NaN,NaN,0.623907,1.000000,0.644480
totals_pageviews,0.006483,0.003720,NaN,0.184333,NaN,NaN,0.987265,0.644480,1.000000


In [238]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import recall_score

import pyspark.pandas as ps
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.feature import *
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import SQLContext

import chart_studio.plotly as py
import plotly.graph_objs as go
import requests
requests.packages.urllib3.disable_warnings()

In [239]:
spark.newSession()

In [240]:
train = train.fillna(0)
train.dtypes, target.dtype

(date                                int64
 fullVisitorId                      uint64
 totals_visits                       int64
 totals_totalTransactionRevenue    float64
 totals_bounces                    float64
 totals_newVisits                  float64
 totals_hits                         int64
 totals_timeOnSite                 float64
 totals_pageviews                  float64
 dtype: object,
 CategoricalDtype(categories=[1.0, 2.0, 3.0, 5.0, 0.0], ordered=False))

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=.3, stratify=target)

In [ ]:
dum_clf = DummyClassifier(strategy='stratified', random_state=RANDOM_SEED)
# lr = LogisticRegression(solver='lbfgs', max_iter=40000)
dtree = DecisionTreeClassifier(criterion='entropy',random_state=RANDOM_SEED)

In [243]:
train_spark = processed[['date','fullVisitorId','totals_visits','totals_totalTransactionRevenue',
               'totals_bounces','totals_newVisits','totals_hits','totals_timeOnSite',
               'totals_pageviews', 'totals_transactions']]

train_spark = spark.createDataFrame(train_spark).withColumn('label', col('totals_transactions'))
train_spark = (train_spark.withColumn('label', when(train_spark['totals_transactions'] == np.nan, 0.0).otherwise(1.0)))
train_spark = train_spark.dropna(subset=['fullVisitorId'])

# train_spark.select('*').filter(col('label')==0.0).head(2)
train_spark.select([(count(when(col(c).contains('None') | \
                              col(c).contains('NULL') | \
                              (col(c) == '') | \
                              col(c).isNull() | \
                              isnan(c) , c)) / count('*')).alias(c) for c in train_spark.columns]).show()


+----+-------------+-------------+------------------------------+------------------+------------------+-----------+------------------+--------------------+-------------------+-----+
|date|fullVisitorId|totals_visits|totals_totalTransactionRevenue|    totals_bounces|  totals_newVisits|totals_hits| totals_timeOnSite|    totals_pageviews|totals_transactions|label|
+----+-------------+-------------+------------------------------+------------------+------------------+-----------+------------------+--------------------+-------------------+-----+
| 0.0|          0.0|          0.0|            0.9883119946814214|0.4987025241802312|0.2068669711981814|        0.0|0.5023697698856935|8.578352527397113E-5| 0.9882691029187844|  0.0|
+----+-------------+-------------+------------------------------+------------------+------------------+-----------+------------------+--------------------+-------------------+-----+



In [ ]:
# X_train_spark = spark.createDataFrame(X_train)
# X_val_spark = spark.createDataFrame(X_val)
# y_train_spark = ps.from_pandas(y_train)
# y_val_spark = ps.from_pandas(y_val)

In [ ]:
for c in train_spark.columns:
  if c != 'fullVisitorId':
    train_spark = train_spark.withColumn(c, 
                                  train_spark[c]
                                  .cast('int'))
train_spark.head()

Row(date=20160801, fullVisitorId=5160807529212499245, totals_visits=1, totals_bounces=0, totals_newVisits=1, totals_hits=10, totals_timeOnSite=148, totals_pageviews=9, totals_transactions=0, label=0)

In [244]:
train_spark = train_spark.drop('totals_totalTransactionRevenue').na.fill({'totals_bounces':0.0, 'totals_newVisits':0.0,
                                                                          'totals_timeOnSite':0.0, 'totals_pageviews':0.0,
                                                                          'totals_transactions':0.0})
X_train_spark, X_val_spark, X_test_spark = train_spark.randomSplit([0.8, 0.1, 0.1], seed=RANDOM_SEED)

In [246]:
spark_lr = LogisticRegression(maxIter=100000, featuresCol='features',labelCol='label')
paramGrid = ParamGridBuilder()\
    .addGrid(spark_lr.regParam, [0.1, 0.01])\
    .build()

cols = [col for col in X_train_spark.columns if (col not in set(['totals_transactions','label','fullVisitorId']))]
vec = VectorAssembler(inputCols=cols, outputCol='features', handleInvalid='keep')

pipeline = Pipeline(stages=[vec, spark_lr])
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=10)

In [247]:
crossval = crossval.fit(X_train_spark)
model1 = crossval.bestModel.stages[1]

In [251]:
prediction = crossval.transform(X_val_spark)
selected = prediction.select("probability", "prediction")

In [252]:
selected.show()

+--------------------+----------+
|         probability|prediction|
+--------------------+----------+
|[0.99302458047730...|       0.0|
|[0.99302458047730...|       0.0|
|[0.98947843405498...|       0.0|
|[0.99302458047730...|       0.0|
|[0.98771898478127...|       0.0|
|[0.99128137003209...|       0.0|
|[0.99302458047730...|       0.0|
|[0.99005282448337...|       0.0|
|[0.99302458789364...|       0.0|
|[0.98061427776625...|       0.0|
|[0.98836979851361...|       0.0|
|[0.99302458789364...|       0.0|
|[0.98523316892570...|       0.0|
|[0.99142543230717...|       0.0|
|[0.98616077706071...|       0.0|
|[0.98959494523042...|       0.0|
|[0.99302458789364...|       0.0|
|[0.99302458789364...|       0.0|
|[0.99302458789364...|       0.0|
|[0.98008128017840...|       0.0|
+--------------------+----------+
only showing top 20 rows

